In [1]:
# 读取人脸数据库olivettifaces,并存储为pkl文件
import numpy as np
from PIL import Image
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import xlwt
import net_input

d:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 读数据

In [2]:
file_name = 'E:/ivy_machine_learning/program/莫烦学习/人脸识别/faces_data.pkl'
dataset, train_data,test_data, train_label,test_label = net_input.read_file(file_name)
batch_size = 20

reading file ...
Reading done.


## 搭建网络

In [5]:
# 方法2
def weight_variable(shape):
    inital = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(inital) 
def bias_variable(shape):
    inital = tf.constant(0.1,shape=shape)##初始值为0.1
    return tf.Variable(inital) 
def conv2d(x,W):
    ##strides=[batch,height,width,channel]，所以固定不动Batch和channel,只改变height和width的步长
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')  
def max_pool_2(x):
    ###kszie难道不是指核的大小？？？不是！！！！
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')  

### （57，47）====》（40）
def model(x_image):
    ## conv1 layer ##
#     print(x_image.shape)
    W_conv1 = weight_variable([5,5,1,32]) ##patch5*5,in_size=1,out_size=32
    b_conv1 = bias_variable([32])
    conv1 = tf.nn.bias_add(conv2d(x_image,W_conv1),b_conv1)  ##output:57*47*32
    conv1_relu = tf.nn.relu(conv1)   ##output:57*47*32
    conv1_pool = max_pool_2(conv1_relu)  ##output:29*24*32   # 向上取整

    ## conv2 layer ##
    W_conv2 = weight_variable([5,5,32,64]) ##patch5*5,in_size=32,out_size=64
    b_conv2 = bias_variable([64])
    conv2 = tf.nn.bias_add(conv2d(conv1_pool,W_conv2),b_conv2)  ##output:29*24*64
    conv2_relu = tf.nn.relu(conv2)   ##output:29*24*64
    conv2_pool = max_pool_2(conv2_relu)  ##output:15*12*64

    ##func1 layer##
    W_fc1 = weight_variable([15*12*64,1024])
    b_fc1 = bias_variable([1024])
    ##shape[n_sample,7,7,64]-->>[n_sample,7*7*64]
    conv2_pool_flat = tf.reshape(conv2_pool,[-1,15*12*64])
    fc1 = tf.matmul(conv2_pool_flat,W_fc1)+b_fc1
    fc1_relu = tf.nn.relu(fc1)
    fc1_relu_drop = tf.nn.dropout(fc1_relu,keep_prob) 
    ##func2 layer##
    W_fc2 = weight_variable([1024,40])
    b_fc2 = bias_variable([40])
    fc2 = tf.matmul(fc1_relu_drop,W_fc2)+b_fc2
    return fc2
def compute_accuracy(v_xs,v_ys):
    global prediction  #将prediction定义为全局变量
    y_pre = sess.run(prediction,feed_dict={X:v_xs,Y:v_ys,keep_prob:0.5})  ##得到1行10列的数据，也就是[1,10]
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    ##取第二维概率最大的（这里索引了最大值的位置），同样标签为最大的值也是我真实的标签值（索引了位置）（我的标签是one_hot类型的）
    ###判断预测和真实标签的index是否一样，一样就代表预测正确，否则就错误
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={X:v_xs,Y:v_ys})
    return result

In [6]:
# 搭建图-----会话图
# workbook = xlwt.Workbook() 
# sheet = workbook.add_sheet("loss") 
j = 1
with tf.Graph().as_default():
#     tf.reset_default_graph() # 这个可以不用细究，是为了防止重复定义报错
    ###define placeholder for inputs to network # 给X、Y定义placeholder，要指定数据类型、形状：
    X = tf.placeholder(tf.float32,[None,2679],name='X')  ##57*47，None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
    Y = tf.placeholder(tf.float32,[None,40],name='Y')  ##40类输出
    keep_prob = tf.placeholder(tf.float32)  ###定义丢掉率
    x_image = tf.reshape(X,[-1,57,47,1])##shape:[n_sample,28,28,1]
    
    global_steps = tf.Variable(1, trainable=False)
    
    # 先定义网络-----获得prediction
    prediction = model(x_image)
    print(prediction.shape)
    # 定义我们的cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=Y))
    # 定义optimizer来minimize cost
    trainer = tf.train.AdamOptimizer().minimize(cost,global_step=global_steps)
    saver = tf.train.Saver()
  
    format_str = "%s    Iteration =  %d   loss = %0.3f"
    with tf.Session() as sess:
        # 首先给所有的变量都初始化（不用管什么意思，反正是一句必须的话）：
        sess.run(tf.global_variables_initializer())

        # 定义一个costs列表，来装迭代过程中的cost，从而好画图分析模型训练进展
        costs = []  

        # 指定迭代次数：
        for it in range(20+1):
            batch = dataset.next_batch(batch_size)
            _,batch_cost = sess.run([trainer,cost],feed_dict={X:batch[0],Y:batch[1],keep_prob:0.5})
#             _,batch_cost = sess.run([trainer,cost],feed_dict={X:train_data,Y:train_label,keep_prob:0.5})
#             loss_new = str(round(batch_cost, 3))
#             sheet.write(it, 1, loss_new)
#             workbook.save('loss_face.xls')

            # 每100个迭代就打印一次cost：
            if it % 10 == 0:
#                 print('iteration%d ,batch_cost: '%it,batch_cost)
                print('loss:',format_str%(datetime.now(),it,batch_cost))
#                 print('正确率：',compute_accuracy(train_data,train_label))
#                 costs.append(batch_cost)

            if it % 20 == 0:
                file = str(j)+"saver_face"
                checkpoint = file+'/' + 'model.ckpt'
                saver.save(sess, checkpoint, global_step=global_steps)  
                
#             if it % 1000 == 0:
#                 write_file=open('E:/ivy_machine_learning/program/莫烦学习/人脸识别/faces_loss2.pkl','wb')  
#                 pickle.dump(costs,write_file)  # 将数据存入pickle
#                 write_file.close() 
        
         ### 训练结束后，看下测试集和训练集上的正确率
#         print('test_accuracy:',compute_accuracy(test_data,test_label))
#         print('train_accuracy:',compute_accuracy(train_data,train_label))

(?, 40)
loss: 2018-12-21 20:43:24.476421    Iteration =  0   loss = 31.150
loss: 2018-12-21 20:43:27.703828    Iteration =  10   loss = 4.764
loss: 2018-12-21 20:43:29.736402    Iteration =  20   loss = 3.683


In [7]:
def train():
    j = 1
    with tf.Graph().as_default():
        X = tf.placeholder(tf.float32,[None,2679],name='X')  ##57*47，None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
        Y = tf.placeholder(tf.float32,[None,40],name='Y')  ##40类输出
        keep_prob = tf.placeholder(tf.float32)  ###定义丢掉率
        x_image = tf.reshape(X,[-1,57,47,1])##shape:[n_sample,28,28,1] 
        global_steps = tf.Variable(1, trainable=False)
        prediction = model(x_image)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=Y))
        trainer = tf.train.AdamOptimizer().minimize(cost,global_step=global_steps)
        saver = tf.train.Saver()
        format_str = "%s    Iteration =  %d   loss = %0.3f"
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for it in range(20+1):
                batch = dataset.next_batch(batch_size)
                _,batch_cost = sess.run([trainer,cost],feed_dict={X:batch[0],Y:batch[1],keep_prob:0.5})
                if it % 10 == 0:
                    print('loss:',format_str%(datetime.now(),it,batch_cost))
                if it % 20 == 0:
                    file = str(j)+"saver_face"
                    checkpoint = file+'/' + 'model.ckpt'
                    saver.save(sess, checkpoint, global_step=global_steps)  
    print('训练结束！')

In [8]:
if __name__ == '__main__':
    train()

ValueError: Tensor("dropout/random_uniform:0", shape=(?, 1024), dtype=float32) must be from the same graph as Tensor("Placeholder:0", dtype=float32).